In [1]:
import random
from pythreejs import *
import numpy as np
from IPython.display import display

In [2]:
scene = Scene(
    background='#000000'
)

camera = PerspectiveCamera(
    position=[10, 10, 10], 
    fov=50, 
    aspect=1.5
)

camera.lookAt([2, 1, 0])

width=800
height=600

renderer = Renderer(
    camera=camera, 
    scene=scene, 
    controls=[OrbitControls(controlling=camera)], 
    width=width, 
    height=height
)

In [3]:
def make_random_tracks(number_of_tracks):

    tracks = []
    
    for i in range(0,number_of_tracks+1):
        pt = 1.0 + random.random()*(50-2.0)
        charge = 1 if random.random() > 0.5 else -1
        eta = ((2*random.random())-1)*2.4
        phi = ((2*random.random())-1)*math.pi

        tracks.append(
            (pt,eta,phi,charge)
        )

    return tracks


def make_helix_points(track):

    pt = track[0]
    eta = track[1]
    phi = track[2]
    charge = track[3]
    
    vertex = (0,0,0)
                                                                                                                                                                                  
    Bz = 3.8
    radius = (pt / (math.fabs(charge)*Bz))

    centerX = vertex[0] + radius*charge*math.sin(phi)
    centerY = vertex[1] - radius*charge*math.cos(phi)
    
    pz = pt*math.sinh(eta)
    phi0 = math.atan2(vertex[1] - centerY, vertex[0] - centerX)
    omega = (charge*Bz) / pt
    pitch = pz / (pt*math.fabs(omega))

    maxAngle = 4*math.pi
    numPoints = 1000
    points = []

    for i in range(0, numPoints+1):

        t = (i/numPoints)*maxAngle
        x = centerX + radius*math.cos(phi0+t)
        y = centerY + radius*math.sin(phi0+t)
        z = vertex[2] + t*pitch
        r = math.sqrt(x*x +y*y)                                                                                                                                                                       
        
        if math.fabs(z) > 3 or r > 1.24:
            break
        
        points.append((float(x),float(y),float(z)))

    points = np.array(points)
    return points

def make_eb():

    geometry = CylinderGeometry(
        radiusTop=1.12, 
        radiusBottom=1.24, 
        height=6.0, 
        radialSegments=64, 
        heightSegments=1, 
        openEnded=True, 
        thetaStart=0, 
        thetaLength=2*math.pi
    )

    eb = Mesh(
        geometry,
        MeshBasicMaterial(
            color='#7fccff',
            wireframe=True,
            transparent=True,
            opacity=0.2
        )
    )

    eb.rotateX(math.pi/2)
    eb.name = 'EB'
    
    return eb

In [6]:
scene.children = []

tracks = make_random_tracks(100)

for track in tracks:
    points = make_helix_points(track)
    points = np.array(points)

    geometry = LineGeometry(positions=points)

    material = LineMaterial(
        linewidth=3,
        color="#ffff00"
    )

    material.resolution = [width, height]

    line = Line2(geometry=geometry, material=material)
    
    scene.add(line)

In [7]:
scene.add(make_eb())
display(renderer)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(-3.5497406696594838, 2.758975741744669, -7.15090620909…